In [16]:
import torch
import numpy as np
import sys
sys.path.append('/media/yujie/data/E2EAD/endtoenddriving')
from navsim.common.dataclasses import AgentInput, Scene, Annotations, Lidar
from navsim.common.enums import BoundingBoxIndex, LidarIndex
from navsim.visualization import lidar

use_ground_plane = False

def _get_lidar_feature(agent_input: AgentInput) -> torch.Tensor:
    """
    Compute LiDAR feature as 2D histogram, according to Transfuser
    :param agent_input: input dataclass
    :return: LiDAR histogram as torch tensors
    """

    # only consider (x,y,z) & swap axes for (N,3) numpy array
    lidar_pc = agent_input.lidars[-1].lidar_pc[LidarIndex.POSITION].T # type: ignore

    # NOTE: Code from
    # https://github.com/autonomousvision/carla_garage/blob/main/team_code/data.py#L873
    def splat_points(point_cloud):
        # 256 x 256 grid
        xbins = np.linspace(
            -32,
            32,
            5,
        )
        ybins = np.linspace(
            -32,
            32,
            5,
        )
        hist = np.histogramdd(point_cloud[:, :2], bins=(xbins, ybins))[0]
        hist[hist > 5] = 5
        overhead_splat = hist / 5
        return overhead_splat

    # Remove points above the vehicle
    lidar_pc = lidar_pc[lidar_pc[..., 2] < 100]
    below = lidar_pc[lidar_pc[..., 2] <= 0.2]
    above = lidar_pc[lidar_pc[..., 2] > 0.2]
    above_features = splat_points(above)
    print(above_features)
    if use_ground_plane:
        below_features = splat_points(below)
        features = np.stack([below_features, above_features], axis=-1)
    else:
        features = np.stack([above_features], axis=-1)
    features = np.transpose(features, (2, 0, 1)).astype(np.float32)

    return torch.tensor(features)

from defusedxml import DTDForbidden
import numpy as np

pcd_file = "/media/yujie/data/E2EAD/endtoenddriving/dataset/sensor_blobs/private_test_e2e/MergedPointCloud/0a0e88dc86ee34d1.pcd"
f = open(pcd_file, 'rb')
# 读取pcd文件头信息
header = "0a0e88dc86ee34d1.pcd"
while True:
    line = f.readline().decode('utf-8')
    header += line
    if line.startswith("DATA"):
        break

# 从文件内容中读取三维坐标点云数据
#dtype = np.dtype([('x', np.float32), ('y', np.float32), ('z', np.float32), ('intensity', np.float32),('ring', np.float32),('lidar_id', np.float32)])
data = np.fromfile(f, dtype=np.float32)
data = data[:30].reshape(5, 6)
print(data)

# Define your lidar data
lidar_pc_data = data

# Create a Lidar object
lidar_ = Lidar(lidar_pc=lidar_pc_data)

# Create an AgentInput instance with a list containing the lidar data
lidar_data = [lidar_]
agent_input = AgentInput(ego_statuses=[], cameras=[], lidars=lidar_data)

# Call the _get_lidar_feature function
lidar_feature = _get_lidar_feature(agent_input)

print(lidar_feature)

print(lidar_feature.size())

lidar_feature_nonzero = np.nonzero(lidar_feature)
print(lidar_feature_nonzero)
#print(lidar_feature[0, 125:128, 125:128])


In [22]:
import numpy as np

# 假设你有一些点云数据和直方图的边界
point_cloud = np.random.rand(5, 3)  # 假设点云数据是一个包含100个点的数组，每个点有3个坐标
print(point_cloud)
above = point_cloud[point_cloud[..., 2] > 0.2]
print(above)
xbins = np.linspace(0, 1, 10)  # 假设x轴的边界
ybins = np.linspace(0, 1, 10)  # 假设y轴的边界

# 使用 numpy.histogramdd 计算直方图
hist = np.histogramdd(above[:, :2], bins=(xbins, ybins))[0]
hist[hist > 5] = 5
overhead_splat = hist / 5
hist_nonzero = np.nonzero(hist)
print(hist)
print(hist_nonzero)


In [2]:
import pickle

path_train = "/media/yujie/data/E2EAD/endtoenddriving/exp/submission_lstm8_ego_state_warmup/2024.05.03.11.25.55/submission.pkl"

f = open(path_train, 'rb')
data_train = pickle.load(f)
# print(f"train: {data_train[1]['log_name']}")
# for i in range(1, len(data_train)):
#     if data_train[i]['log_name'] != data_train[i-1]['log_name']:
#         print(f"train: {data_train[i]['log_name']}")
#     #print(data[i]['scene_name'])

In [49]:
from defusedxml import DTDForbidden
import numpy as np

pcd_file = "/media/yujie/data/E2EAD/endtoenddriving/dataset/sensor_blobs/private_test_e2e/MergedPointCloud/0a0e88dc86ee34d1.pcd"
f = open(pcd_file, 'rb')
# 读取pcd文件头信息
header = "0a0e88dc86ee34d1.pcd"
while True:
    line = f.readline().decode('utf-8')
    header += line
    if line.startswith("DATA"):
        break

# 从文件内容中读取三维坐标点云数据
#dtype = np.dtype([('x', np.float32), ('y', np.float32), ('z', np.float32), ('intensity', np.float32),('ring', np.float32),('lidar_id', np.float32)])
data = np.fromfile(f, dtype=np.float32)

#print(data)  
#data = np.fromfile(f, dtype=dtype)
print(data[-1])


In [2]:
import torch

a = {'a': torch.tensor([[ 664651.3614, 3999258.5894],
                [ 664650.9561, 3999257.7967],
                [ 664650.6527, 3999257.2051],
                [ 664650.4523, 3999256.8068],
                [ 664650.3505, 3999256.6042],
                [ 664650.3300, 3999256.5648],
                [ 664650.3476, 3999256.5846],
                [ 664650.3563, 3999256.5980]], dtype=torch.float64)}
b = 'train'
c = [a,b]
# print(c)
# print(c[0])
# print(c[1])
b = torch.randn(1, 8, 11)
current_feature = torch.zeros(1, 1, 3)
print(b)
b[:, :, :3] = torch.cat((b[:, :-1, :3] - b[:, 1:, :3], current_feature), dim=1)
print(b)
# zero_row = torch.zeros_like(a[-1]).unsqueeze(0)
# print(zero_row)
# torch.cat((a[:-1] - a[1:], zero_row), dim=0)
#b = torch.tensor([])

In [ ]:
path_val = "/home/yujie/OpenScene/dataset/data/openscene_mini_val.pkl"

f = open(path_val, 'rb')
data_val = pickle.load(f)
print(f"val: {data_val[1]['log_name']}")
for i in range(1, len(data_val)):
    if data_val[i]['log_name'] != data_val[i-1]['log_name']:
        print(f"val: {data_val[i]['log_name']}")

In [34]:

import pickle
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np

path = "/media/yujie/data/E2EAD/endtoenddriving/dataset/navsim_logs/private_test_e2e/competition_test.pkl"
# /media/yujie/data/E2EAD/endtoenddriving/dataset/navsim_logs/private_test_e2e/competition_test.pkl
f = open(path, 'rb')
data = pickle.load(f)
i = 10
print("Path, i: ", path, i)
print("Time stamp: ", data[i]['timestamp'])
print("Driving command: ", data[i]['driving_command'])
print("The velocity and acceleration of ego car: ", data[i]['ego_dynamic_state'])

info_CAM_B0 = data[i]['cams']['CAM_B0']
info_CAM_F0 = data[i]['cams']['CAM_F0']
info_CAM_L0 = data[i]['cams']['CAM_L0']
info_CAM_L1 = data[i]['cams']['CAM_L1']
info_CAM_L2 = data[i]['cams']['CAM_L2']
info_CAM_R0 = data[i]['cams']['CAM_R0']
info_CAM_R1 = data[i]['cams']['CAM_R1']
info_CAM_R2 = data[i]['cams']['CAM_R2']

path_sensor = "/media/yujie/data/E2EAD/endtoenddriving/dataset/sensor_blobs/private_test_e2e"

# 构建图像路径字典
image_paths = {
    'B0': info_CAM_B0['data_path'],
    'F0': info_CAM_F0['data_path'],
    'L0': info_CAM_L0['data_path'],
    'L1': info_CAM_L1['data_path'],
    'L2': info_CAM_L2['data_path'],
    'R0': info_CAM_R0['data_path'],
    'R1': info_CAM_R1['data_path'],
    'R2': info_CAM_R2['data_path']
}

# 使用循环读取图像并转换为RGB格式
images_rgb = {}
for name, path in image_paths.items():
    image = cv2.imread(os.path.join(path_sensor, path))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images_rgb[name] = image_rgb

# 创建一张白图
white_image = np.ones_like(images_rgb['F0']) * 255  # Create a white image

# 显示图像
fig, axs = plt.subplots(3, 3, figsize=(9, 9))

# 左视图
axs[0, 0].imshow(images_rgb['L0'])
axs[0, 0].set_title('Left 0')
axs[0, 0].axis('off')

axs[1, 0].imshow(images_rgb['L1'])
axs[1, 0].set_title('Left 1')
axs[1, 0].axis('off')

axs[2, 0].imshow(images_rgb['L2'])
axs[2, 0].set_title('Left 2')
axs[2, 0].axis('off')

# 前视图和白图
axs[0, 1].imshow(images_rgb['F0'])
axs[0, 1].set_title('Front')
axs[0, 1].axis('off')

axs[1, 1].imshow(white_image, cmap='gray')
axs[1, 1].set_title('')
axs[1, 1].axis('off')

axs[2, 1].imshow(images_rgb['B0'])
axs[2, 1].set_title('Back')
axs[2, 1].axis('off')

# 右视图
axs[0, 2].imshow(images_rgb['R0'])
axs[0, 2].set_title('Right 0')
axs[0, 2].axis('off')

axs[1, 2].imshow(images_rgb['R1'])
axs[1, 2].set_title('Right 1')
axs[1, 2].axis('off')

axs[2, 2].imshow(images_rgb['R2'])
axs[2, 2].set_title('Right 2')
axs[2, 2].axis('off')

plt.tight_layout()
plt.show()

In [36]:
# Anns
anns_boxes = data[i]['anns']['gt_boxes'] # X = 0,_Y = 1,_Z = 2,_LENGTH = 3,_WIDTH = 4,_HEIGHT = 5,_HEADING = 6
print(anns_boxes)

In [ ]:
import torch
from torch import tensor
x = tensor([[ 664754.0141, 3999465.8250],
        [ 664754.0147, 3999465.8251],
        [ 664754.0146, 3999465.8251],
        [ 664754.0146, 3999465.8251],
        [ 664754.0146, 3999465.8250],
        [ 664754.0147, 3999465.8251],
        [ 664754.0147, 3999465.8250],
        [ 664754.0147, 3999465.8250],
        [ 664754.0146, 3999465.8250],
        [ 664754.0146, 3999465.8250],
        [ 664754.0146, 3999465.8250],
        [ 664754.0146, 3999465.8250],
        [ 664754.0146, 3999465.8250],
        [ 664754.0147, 3999465.8250],
        [ 664754.0146, 3999465.8251],
        [ 664754.0142, 3999465.8251]], dtype=torch.float64)
#print(x[:-1])
y = tensor([[[1, 2],
             [2, 3],
             [3, 4],
             [4, 5]],
            [[6, 1],
             [7, 2],
             [8, 3],
             [9, 0]]], dtype=torch.float32)
z = tensor([[[-1, 2],
             [2, 3],
             [3, 4],
             [4, 5]],
            [[-6, 1],
             [7, 2],
             [8, 3],
             [9, 0]]], dtype=torch.float32)

delta_mul = torch.mul(y, z)

# 遍历 y 的每一行，计算累加和并填充到 delta_sum 中
# for i in range(1, y.size(0)):
#     delta_sum[:,:,i] = y[:,:,:i].sum(dim=0)
delta_sum[:, 1:, :] = torch.cumsum(y[:, :-1, :], dim=1)

print(delta_mul)
print(torch.cumsum(y, dim=1))

#print(x[:-1] - x[1:])

In [ ]:
# import torch
# import torch.nn as nn
# from torch import Tensor
# # test
# batch_size = 2
# x = Tensor([[1, 2, 3, 4, 5, 6],[1, 2, 3, 4, 5, 6]])
# print(x.size())
# x = x.view(batch_size, -1, 2)
# print(x)


In [ ]:
i = 24
info_CAM_B0 = data[i]['cams']['CAM_B0']
info_CAM_F0 = data[i]['cams']['CAM_F0']
info_CAM_L0 = data[i]['cams']['CAM_L0']
info_CAM_L1 = data[i]['cams']['CAM_L1']
info_CAM_L2 = data[i]['cams']['CAM_L2']
info_CAM_R0 = data[i]['cams']['CAM_R0']
info_CAM_R1 = data[i]['cams']['CAM_R1']
info_CAM_R2 = data[i]['cams']['CAM_R2']

# print(data[0]['ego2global_rotation'])
# print(data[0]['timestamp'])
# print(data[1]['timestamp'])
# print(data[2]['timestamp'])

In [ ]:
import open3d as o3d

path_pointcloud = os.path.join(path_sensor, data[0]['lidar_path'])

pointcloud = o3d.io.read_point_cloud(path_pointcloud)

# 可视化点云
o3d.visualization.draw_geometries([pointcloud])

In [ ]:
data[0]['anns']['track_tokens']